![](../images/dtlogo.png)

# Object detection for robots

## Integration

Finally, we need to integrate our model with ROS. Editing the functions in this notebook will edit the object detection node found in `src/object_detection/src`.


### Simulation 💻
If you don't have a Jetson Nano Duckiebot, you can run this exercise locally with the simulator. This will run your code as a Pytorch model. This means that we rely on your host machine's CPU or GPU to run your model.

### Hardware 🚙

If you have a Jetson Nano Duckiebot, you can run this exercise on your Duckiebot. This will convert your model to a TensorRT model, and run it your Jetson Nano's GPU.

### What we are doing in this notebook

In both cases, we need to edit the ROS node to decide how we will use the detections.
Should you call your model on every image from your camera? Only once every 4-5 frames, to preserve your CPU?
You might also want to change your robot's behaviour depending on the size of the detection.
If it is small, the object is probably far away, so there's no need to stop.

To get started, input your information in the functions bellow.

In [1]:
def DT_TOKEN():
    # todo change this to your duckietown token
    dt_token = "dt1-3nT8KSoxVh4MdLnE1Bq2mTkhRpbR35G8mmbjExKF7zGm6g4-43dzqWFnWd8KBa1yev1g3UKnzVxZkkTbfYtfGWrfSxeihNZvYVNfNmnCBP28LeqDxL"
    return dt_token

def MODEL_NAME():
    # todo change this to your model's name that you used to upload it on google colab.
    # if you didn't change it, it should be "yolov5"
    return "yolov5"

Then, you can use `dts exercises build` and `dts exercises test` (either with `--sim` for in simulation or with `-b DUCKIEBOT_NAME` for the real Duckiebot) to see it in action.


### Framerate

While object detection is useful, it is also very expensive, computationally.

One trick used to reduce said cost is to only call the full model infrequently.
For example, one might call the model only a few times a second, which is very slow in
computer timeframes, but relatively fast for the real world. 

Of course, this varies from application to application. In very dynamic, fast
robotic environments, clearly the model should be called more frequently.

You can fine-tune this yourself: the function bellow indicates the number of frames
your robot should skip before calling its object detection model.

In [2]:
def NUMBER_FRAMES_SKIPPED():
    # todo change this number to drop more frames
    # (must be a positive integer)
    return 0

In real life, we would use a full neural network model to produce very accurate
predictions, and then a less accurate model coupled with a Kalman filter (or
other such estimation system) to "track" each prediction during the skipped frames.

For this exercises, we will limit ourselves to just skipping frames.

### Filtering

Some of your predictions might not actually be useful. For example, in Duckietown,
the trucks and busses are always parked on the side of the road. Your robot will
never have to avoid or stop for them.

Cones can be in the road in some maps, but for this exercises, you can assume that there
aren't any.

#### Filtering by class

For this reason, you probably want to remove all non-duckies from your predictions,
since only duckies will be on the road.

In [3]:
# `class` is the class of a prediction
def filter_by_classes(clas):
    # Right now, this returns True for every object's class
    # Change this to only return True for duckies!
    # In other words, returning False means that this prediction is ignored.
    return True

#### Filtering by score

Depending on the model, you might also want to remove very unconfident detections.

Then again, your model might not be confident even for detections that are absolutely
correct. You should experiment to find a value that works well for your model.

In [4]:
# `scor` is the confidence score of a prediction
def filter_by_scores(scor):
    # Right now, this returns True for every object's confidence
    # Change this to filter the scores, or not at all
    # (returning True for all of them might be the right thing to do!)
    return True

#### Filtering by bounding box

Finally, you should also evaluate what each detection *means* in terms of positionning.

If a bounding box is in the leftmost or rightmost thirds of the image, it might be the case that
the object in not even on the road, and that your robot would be able to go by it without issue.

![image of a bounding box](../images/thirds.png)

Also, if a bounding box's area is small, its object is likely far away. So there is no need to
try to avoid the object or stop the robot: the robot still has a bit of driving to do before it reaches
the object. So filtering out small detections might be a good idea too.

In [5]:
# `bbox` is the bounding box of a prediction, in xyxy format
# So it is of the shape (leftmost x pixel, topmost y pixel, rightmost x pixel, bottommost y pixel)
def filter_by_bboxes(bbox):
    # Like in the other cases, return False if the bbox should not be considered.
    return True


### Fine-tuning

In all of the functions above, there is not objective right answer. You should play with
your functions and fine-tune their behaviours. Robotics is an iterative process!

# Testing

As mentioned, you can test the behavior of your agent in the simulator with 

    $ dts exercises test --sim

Note that you will need to open the Image Viewer, and then the keyboard control joystick. Once the joystick is up and running, press "a" on it to enable automatic lane following.

You can test on real hardware with

    $ dts exercises test -b ![DUCKIEBOT_NAME]
   
In the case of the real hardware, you will need to be patient. Because of RAM limitations on the Jetson, we need to turn the camera off, then load your model, and then restart the camera. As soon as your model is loaded your robot might start moving! So make sure it is in a safe space. 

## Debugging

After you run `dts exercises test` (either sim or on hardware) you can open the [VNC browser](http://localhost:8087/) to see what is happening. If you click on the `RQT Image View` icon and then in the dropdown select `/![DUCKIEBOT_NAME]/object_detection_node/object_detections_img`. This shows an image with your detections overlayed as shown below:

![](../images/rqt_with_duckie.png)


# Submission

You can test your agent locally with

    $ dts challenges evaluate
    

And then finally submit with 

    $ dts challenges submit
    

And then check out how you did [on the leaderboard](https://challenges.duckietown.org/v4/humans/challenges/mooc-objdet/leaderboard). 
